# Case Study 1 : Collecting Data from Twitter

Due Date: September 21, **BEFORE the beginning of class at 6:00pm**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    Tingting Ma
    
    Jiani Gao
    
    Jinyan Lyu
    
    Tianhao Guo
    
    Mo Cheng
    

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [10]:
import twitter
import json


# ---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information
    # on Twitter's OAuth implementation.

    CONSUMER_KEY = 'PoOx77O7x6QGWSZJUyPdXK9ci'
    CONSUMER_SECRET = 'ao8VoGR1WwOkjU7ASXQBfTD8IFL9P50R6CzlCW0tFqc1Vx6ifl'
    OAUTH_TOKEN = '354812054-MNiHp7kIJj8gMX1qKkWR7rd5VogbzIPNiGRQW1uD'
    OAUTH_TOKEN_SECRET = '1WbpfZfexiwxHMt804MQI98iBpYeBrCKaiIAXjlBvpZmH'

    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)

    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api


# ----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

def tweet_stream(q, max):
    auth = oauth_login().auth
    # open a json file to save tweets
    file = open(q + '.json', 'w')
    twitter_stream = twitter.TwitterStream(auth=auth)
    # use the twitter stream api
    iterator = twitter_stream.statuses.filter(track=q)
    tweets = []
    for tweet in iterator:
        # check the number of tweets
        if len(tweets) >= max:
            break
        tweets.append(tweet)
    # write data to file
    data = json.dumps(tweets, indent=1)
    file.write(data)
    file.close()
    with open(q + '.json', 'r') as f:
        d = json.loads(f.read())
        print('Number of collected tweets: %d' % len(d))


tweet_stream('September11', 1000)


Number of collected tweets: 1000


### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [14]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

from prettytable import PrettyTable
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english') + list(string.punctuation) + ['https', 'http', '//t.…', "''", '``', "'s"]

def count_words(data):
    words_count = {}
    for tweet in data:
        sentences = tweet['text']
        words = word_tokenize(sentences)
        for word in words:
            if word.lower() not in stop_words:
                words_count[word] = words_count.get(word, 0) + 1
    return words_count

all_tweets = load_json('September11.json')
count = count_words(all_tweets)
sorted_count = sorted(count.items(), key=lambda d:d[1], reverse=True)

frequency_pt = PrettyTable(field_names = ['word', 'frequency'])
[frequency_pt.add_row(kv) for kv in sorted_count[:30]]
print(frequency_pt)







+----------------+-----------+
|      word      | frequency |
+----------------+-----------+
|       RT       |    809    |
|  September11   |    647    |
|  NeverForget   |    354    |
|      lost      |    175    |
|    remember    |    171    |
|     lives      |    138    |
|       16       |    130    |
|      amp       |    124    |
|     years      |    123    |
|     Today      |    114    |
|      ago       |    103    |
|      9/11      |     91    |
|       🇺🇸       |     80    |
|      day       |     71    |
|      2001      |     71    |
|  neverforget   |     59    |
|   September    |     58    |
|     Trump      |     56    |
|      11th      |     55    |
| HistoryToLearn |     54    |
|     today      |     54    |
|       us       |     54    |
|     never      |     51    |
|   responders   |     49    |
|     United     |     47    |
|     stand      |     47    |
|     honor      |     47    |
|     first      |     44    |
|      much      |     44    |
|     fo

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [15]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

def count_sort_retweets(statuses):
    retweets_count = {}
    for tweet in statuses:
        if 'retweeted_status' in tweet.keys() and 'retweet_count' in tweet["retweeted_status"].keys():
            if tweet['text'] not in retweets_count.values():
                retweets_count[tweet['retweeted_status']['retweet_count']] = tweet['text']
    sorted_retweets_count = sorted(retweets_count.items(), key=lambda d:d[0], reverse=True)
    return sorted_retweets_count

tweets_data = load_json('September11.json')
retweets_count = count_sort_retweets(tweets_data)
for item in retweets_count[:10]:
    print("retweeted_number: ", item[0], "\t","tweet_text: ", item[1])



retweeted_number:  8198 	 tweet_text:  RT @mark_wahlberg: United we stand. #neverforget #September11 https://t.co/YzEMh6rkRd
retweeted_number:  4613 	 tweet_text:  RT @BillClinton: We are united in remembering those we lost, their families, and the heroic first responders in NY, VA, PA, and across Amer…
retweeted_number:  4014 	 tweet_text:  RT @HistoryToLearn: Today we remember the lives that were lost, 16 years ago, on September 11th, 2001. #NeverForget #September11 https://t.…
retweeted_number:  3972 	 tweet_text:  RT @MilitaryEarth: It's been 16 years, and I'll never forget! Let's all take a moment to remember this day. 🇺🇸 #September11… 
retweeted_number:  3026 	 tweet_text:  RT @AlwaysActions: #EricBolling: "Stand up and
show some respect for your
country, or get the hell out"

#MAGA #AmericaFirst 🚨🇺🇸… 
retweeted_number:  2514 	 tweet_text:  RT @AmirAminiMD: RIP to all terror victims who lost their lives on #September11.
Also RIP to 1.1M Iraqis who had absolutely nothing to do w…


**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [16]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


from prettytable import PrettyTable

def count_sort_hashtags_and_user_mentions(statuses):
    hashtags_count = {}
    user_mentions_count = {}
    for tweet in statuses:
        if 'entities' in tweet.keys() and len(tweet['entities']['hashtags']) != 0:
            for hashtag in tweet['entities']['hashtags']:
                hashtags_count[hashtag['text']] = hashtags_count.get(hashtag['text'], 0) + 1
        if 'entities' in tweet.keys() and len(tweet['entities']['user_mentions']) != 0:
            for user_mention in tweet['entities']['user_mentions']:
                user_mentions_count[user_mention['screen_name']] = user_mentions_count.get(user_mention['screen_name'], 0) + 1
                
    sorted_hashtags_count = sorted(hashtags_count.items(), key=lambda d:d[1], reverse=True)
    sorted_user_mentions_count = sorted(user_mentions_count.items(), key=lambda d:d[1], reverse=True)
    return sorted_hashtags_count, sorted_user_mentions_count

tweets_data = load_json('September11.json')
hashtags_count, user_mentions_count = count_sort_hashtags_and_user_mentions(tweets_data)

hashtags_pt = PrettyTable(field_names = ['hashtag', 'frequency'])
[hashtags_pt.add_row(kv) for kv in hashtags_count[:10]]
user_mentions_pt = PrettyTable(field_names = ['user mentions', 'frequency'])
[user_mentions_pt.add_row(kv) for kv in user_mentions_count[:10]]

print(hashtags_pt)
print(user_mentions_pt)




+------------------+-----------+
|     hashtag      | frequency |
+------------------+-----------+
|   September11    |    679    |
|   NeverForget    |    391    |
|   neverforget    |     63    |
|  September11th   |     29    |
|  NeverForget911  |     22    |
|   september11    |     19    |
| MondayMotivation |     17    |
|  911remembrance  |     14    |
|     Respect      |     13    |
|   Remember911    |     12    |
+------------------+-----------+
+-----------------+-----------+
|  user mentions  | frequency |
+-----------------+-----------+
|  HistoryToLearn |     54    |
|  mark_wahlberg  |     39    |
|      AmyMek     |     37    |
| realDonaldTrump |     28    |
|  MilitaryEarth  |     26    |
|      funder     |     26    |
|       FBI       |     17    |
|  MAGGunslinger  |     16    |
|       Mets      |     16    |
|      POTUS      |     14    |
+-----------------+-----------+


* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary







* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary








*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Yingnan Liu (yliu18@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS501 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

